In [49]:
import numpy as np
from periodic_system import periodic_system
import subprocess

In [28]:
data = np.load("md17_benzene2017.npz", allow_pickle=True)
data_dict = {}
for key in list(data.keys()):
    data_dict[key] = data[key]
print(list(data.keys()))

['E', 'name', 'F', 'theory', 'R', 'z', 'type', 'md5']


In [29]:
data_dict["z"]

array([6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [60]:
def create_molpro_training_data(R, filename, calculation_type):
    rows = []
    num_atoms = len(data_dict["z"])
    for i in range(num_atoms):
        atom_number = data_dict["z"][i]
        atom = periodic_system[atom_number]
        coords = [str(coord) for coord in R[i]]
        row = [atom] + coords
        row = ",".join(row)
        rows.append(row)
    rows = ["geomtyp=xyz"] + ["geometry={"] + [str(num_atoms)] + [""] + rows + ["}"] + ["hf"] + [f"{calculation_type}"]
    rows += ["{matrop"] 
    rows += ["load,Dscf,DEN,TYPE=2"] 
    rows += ["write,Dscf,denfile"]
    rows += ["}"]
    rows = "\n".join(rows)
    f = open(f"{filename}.inp", "w")
    f.write(rows)
    f.close()
    return
    

In [61]:
R = data_dict["R"][0]
filename = "test_data"
create_molpro_training_data(R, filename, "MP2")

**Run molpro**

In [59]:
command = f"/usr/local/molpro/molpro_2022.3/bin/molpro {filename}.inp"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()